In [1]:
import sys
import os
from pathlib import Path

current_dir = str(Path(os.getcwd()).parent.parent.parent)
print(f"Current dir: {current_dir}")
sys.path.insert(0, current_dir)

Current dir: d:\Programming\Python\comfyui-image-scorer


In [ ]:
from shared.paths import filtered_data
from shared.loaders.training_loader import training_loader
from shared.training.data_transformer import data_transformer
import numpy as np

retrain = False
if retrain:
    training_loader.remove_training_models()

print("--- Step 1: Loading Data ---")
if not os.path.exists(filtered_data):
    x = training_loader.load_vectors()
else:
    x = np.empty(0)
y = training_loader.load_scores()


print(f"Loaded Data Shape: {y.shape}")

print("\n--- Step 2: Filtering Unused Features ---")
# Removes features with zero variance or zero importance in a quick probe
x, kept_indices = data_transformer.filter_unused_features(x, y, 500)
print(f"Filtered Data Shape: {x.shape}")

# print("\n--- Step 3: Generating Interaction Features ---")
# Adds top 500 polynomial interactions (feature_A * feature_B)
#x, _ = data_transformer.add_interaction_features(x, y, target_k=1000)
# print(f"Final Data Shape (with Interactions): {x.shape}")
print("Data Preparation Complete.")

--- Step 1: Loading Data ---
Loaded Data Shape: (1000,)

--- Step 2: Filtering Unused Features ---


Training LightGBM: 100%|██████████| 500/500 [00:53<00:00,  9.38it/s]

Dropped 346 unused features. New shape: (1000, 2912)


Training LightGBM: 100%|██████████| 500/500 [00:53<00:00,  9.30it/s]

Saved filtered data to cache
Filtered Data Shape: (1000, 2912)

--- Step 3: Generating Interaction Features ---
Final Data Shape (with Interactions): (1000, 2912)
Data Preparation Complete.


In [3]:
import random
from IPython.display import clear_output
from shared.config import config
from external_modules.step03training.full_data.run import (
    optimize_hyperparameters,
    evaluate_hyperparameter_combo,
)
from external_modules.step03training.full_data.config_utils import (
    crossover_config,
    generate_random_config,
    # generate_slowest_setup
)
from shared.paths import models_dir
from typing import Dict, Any

# Setup Loop Variables
current_cfg = generate_random_config()
top_cfg = config["training"]["top"]
fastest_cfg = config["training"]["fastest"]
slowest_cfg = config["training"]["slowest"]
# slowest_cfg=generate_slowest_setup()
# config["training"]["slowest"]= slowest_cfg
max_iters = config["training"]["max_iters"]
max_combos = config["training"]["max_combos"]
print(f"Initialized HPO loop with max_iters={max_iters}, max_combos={max_combos}")

temp_model_base = os.path.join(models_dir, "temp_model")

get_base_score = True
if get_base_score:
    for value in ["top", "fastest", "slowest"]:
        print(f"getting {value} base score ...")
        current = config["training"][value]
        score, t_time = evaluate_hyperparameter_combo(
            current, temp_model_base, X=x, y=y
        )
        new_current:Dict[str,Any] = {**current, "best_score": score, "training_time": t_time}
        current.update(new_current)
        print(f"score: {score}")


for i in range(max_iters):
    if i % 10 == 0 and i > 0:
        clear_output(wait=True)
    # Refresh references
    top_cfg = config["training"]["top"]
    fastest_cfg = config["training"]["fastest"]
    slowest_cfg = config["training"]["slowest"]

    max_combos = config["training"]["max_combos"]

    # Strategy selection
    rand_val = random.random()
    if rand_val < 0.1:
        base_cfg = generate_random_config()
        strategy = "RANDOM_START"
    elif rand_val < 0.3:
        base_cfg = fastest_cfg
        strategy = "FASTEST"
    elif rand_val < 0.8:
        base_cfg = top_cfg
        strategy = "TOP"
    elif rand_val < 0.9:
        base_cfg = slowest_cfg
        strategy = "SLOWEST"
    else:
        # Crossover
        candidates = [
            c for c in [top_cfg, fastest_cfg, slowest_cfg] if c["best_score"] > -9999
        ]
        if len(candidates) < 2:
            candidates = [c for c in [top_cfg, fastest_cfg, current_cfg] if c]
            if len(candidates) < 2:
                candidates = [generate_random_config(), generate_random_config()]

        parents = random.sample(candidates, 2)
        base_cfg = crossover_config(dict(parents[0]), dict(parents[1]))
        strategy = "CROSSOVER"

    print(f"\nIter {i + 1}/{max_iters} — Strategy: {strategy}")
    print(f"current score; {base_cfg["best_score"]}")
    # optimize_hyperparameters handles updates and saving internally now.
    results = optimize_hyperparameters(
        base_cfg=base_cfg, max_combos=max_combos, X=x, y=y, strategy=strategy
    )

    # Info only loop
    for candidate_cfg, metrics in results:
        r2 = metrics["r2"]
        t_time = metrics["training_time"]

        # Check if this result matches current bests (just for log/info)
        is_top = r2 == top_cfg["best_score"]
        is_fastest = t_time == fastest_cfg["training_time"]

        if is_top:
            print(f"  [Info] This batch produced the current TOP score: {r2:.6f}")
        if is_fastest:
            print(
                f"  [Info] This batch produced the current FASTEST time: {t_time:.4f}s"
            )

    # Refresh local current_cfg for next RANDOM_START or Crossover inheritance
    if results:
        # Simply take the last one as 'current' for random drift
        current_cfg = results[-1][0]

print(f"\nFinished optimization.")
print(f"Top R2: {top_cfg['best_score']:.6f}")
print(
    f"Fastest Time: {fastest_cfg['training_time']:.6f}s (R2: {fastest_cfg['best_score']:.6f})"
)
print(
    f"Slowest (High Score) Time: {slowest_cfg['training_time']:.6f}s (R2: {slowest_cfg['best_score']:.6f})"
)


Iter 11/20 — Strategy: TOP
current score; -4.029479003729996
last used keys for TOP: ['n_estimators', 'num_leaves', 'min_split_gain', 'early_stopping_rounds', 'learning_rate', 'colsample_bytree', 'max_depth', 'subsample']
Optimizing hyperparameters over grid: {'reg_alpha': [3.500387556747282, 2.863953455520503], 'min_child_samples': [333], 'reg_lambda': [6.979543712860458], 'n_estimators': [201], 'num_leaves': [535], 'min_split_gain': [0.37688625564163836], 'early_stopping_rounds': [63], 'learning_rate': [0.006370049059008924], 'colsample_bytree': [0.5075478025511719], 'max_depth': [28], 'subsample': [0.8720896667139454]}
______________________________
Evaluating hyperparameter combo 1/2, with params: {'reg_alpha': 3.500387556747282, 'min_child_samples': 333, 'reg_lambda': 6.979543712860458, 'n_estimators': 201, 'num_leaves': 535, 'min_split_gain': 0.37688625564163836, 'early_stopping_rounds': 63, 'learning_rate': 0.006370049059008924, 'colsample_bytree': 0.5075478025511719, 'max_dept

Training LightGBM:  32%|███▏      | 64/201 [00:00<00:00, 411.51it/s]


r2=-4.0295, time=0.1558s for Evaluated params {'reg_alpha': 3.500387556747282, 'min_child_samples': 333, 'reg_lambda': 6.979543712860458, 'n_estimators': 201, 'num_leaves': 535, 'min_split_gain': 0.37688625564163836, 'early_stopping_rounds': 63, 'learning_rate': 0.006370049059008924, 'colsample_bytree': 0.5075478025511719, 'max_depth': 28, 'subsample': 0.8720896667139454}
______________________________
Evaluating hyperparameter combo 2/2, with params: {'reg_alpha': 2.863953455520503, 'min_child_samples': 333, 'reg_lambda': 6.979543712860458, 'n_estimators': 201, 'num_leaves': 535, 'min_split_gain': 0.37688625564163836, 'early_stopping_rounds': 63, 'learning_rate': 0.006370049059008924, 'colsample_bytree': 0.5075478025511719, 'max_depth': 28, 'subsample': 0.8720896667139454}


Training LightGBM:  32%|███▏      | 64/201 [00:00<00:00, 396.78it/s]


r2=-4.0295, time=0.1616s for Evaluated params {'reg_alpha': 2.863953455520503, 'min_child_samples': 333, 'reg_lambda': 6.979543712860458, 'n_estimators': 201, 'num_leaves': 535, 'min_split_gain': 0.37688625564163836, 'early_stopping_rounds': 63, 'learning_rate': 0.006370049059008924, 'colsample_bytree': 0.5075478025511719, 'max_depth': 28, 'subsample': 0.8720896667139454}
  [Info] This batch produced the current TOP score: -4.029479
  [Info] This batch produced the current TOP score: -4.029479

Iter 12/20 — Strategy: CROSSOVER
current score; -1000000.0
last used keys for CROSSOVER: []
Optimizing hyperparameters over grid: {'min_split_gain': [0.012100000000000001, 0.01], 'n_estimators': [783], 'num_leaves': [891], 'max_depth': [72], 'reg_alpha': [0.04321393885750593], 'learning_rate': [0.018142654163951798], 'subsample': [0.13918990509652804], 'min_child_samples': [52], 'early_stopping_rounds': [162], 'colsample_bytree': [0.1191695216418353], 'reg_lambda': [9.0]}
_______________________

...

r2=-4.0295, time=0.1605s for Evaluated params {'min_child_samples': 366, 'n_estimators': 201, 'num_leaves': 535, 'min_split_gain': 0.37688625564163836, 'early_stopping_rounds': 63, 'learning_rate': 0.006370049059008924, 'colsample_bytree': 0.5075478025511719, 'max_depth': 28, 'subsample': 0.8720896667139454, 'reg_alpha': 3.1821705061338923, 'reg_lambda': 6.979543712860458}
______________________________
Evaluating hyperparameter combo 2/2, with params: {'min_child_samples': 299, 'n_estimators': 201, 'num_leaves': 535, 'min_split_gain': 0.37688625564163836, 'early_stopping_rounds': 63, 'learning_rate': 0.006370049059008924, 'colsample_bytree': 0.5075478025511719, 'max_depth': 28, 'subsample': 0.8720896667139454, 'reg_alpha': 3.1821705061338923, 'reg_lambda': 6.979543712860458}


Training LightGBM:  32%|███▏      | 64/201 [00:00<00:01, 82.14it/s]


r2=-4.0295, time=0.7793s for Evaluated params {'min_child_samples': 299, 'n_estimators': 201, 'num_leaves': 535, 'min_split_gain': 0.37688625564163836, 'early_stopping_rounds': 63, 'learning_rate': 0.006370049059008924, 'colsample_bytree': 0.5075478025511719, 'max_depth': 28, 'subsample': 0.8720896667139454, 'reg_alpha': 3.1821705061338923, 'reg_lambda': 6.979543712860458}
  [Info] This batch produced the current TOP score: -4.029479
  [Info] This batch produced the current TOP score: -4.029479

Iter 15/20 — Strategy: CROSSOVER
current score; -1000000.0
last used keys for CROSSOVER: ['min_split_gain']
Optimizing hyperparameters over grid: {'early_stopping_rounds': [178, 145], 'num_leaves': [891], 'n_estimators': [1800], 'reg_alpha': [6.842495885918459], 'subsample': [0.26397680124976103], 'min_child_samples': [46], 'colsample_bytree': [0.1191695216418353], 'learning_rate': [0.09988797979015301], 'reg_lambda': [0.39447162368857014], 'max_depth': [4], 'min_split_gain': [0.011000000000000

Training LightGBM:   0%|          | 0/1800 [00:00<?, ?it/s]

KeyboardInterrupt: 